<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" style="height:400px;" width=500px/></p>

<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

<h3 style="text-align: center;"><b>Домашнее задание: сегментация изображений</b></h3>

---

<h3 style="text-align: center;"><b>Условие</b></h3>

В качестве домашней работы предлагается улучшить сегментацию, сделанную на семинаре. В качестве основы для даталоэдера используйте функцию __keras_generator__. Попробуйте сделать следующие улучшения:
1. Брать случайный кроп, а не ресайзить исходное изображение. Попробуйте разные размеры кропов, на каких результат получается лучше? Кроп какого максимального размера помещается в вашу видеокарту?
2. Добавить аугментаций. Например, можно поворачивать картинки или добавлять случайный шум

Эти два пункта улучшения можно сделать с помощью библиотеки __albumentations__ https://github.com/albu/albumentations . Пример использования можно посмотреть в конце семинарского ноутбука.

In [ ]:
def strong_aug(p=1.0):
    return Compose([
        ShiftScaleRotate(shift_limit=0.125, scale_limit=0.2, rotate_limit=10, p=0.7, border_mode=cv2.BORDER_CONSTANT),
        RandomCrop(256, 256),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.3),

    ],
        p=p)

augmentation = strong_aug(p=1.0)


Подумайте, нужно ли применять аугментации для валидационной выборки или это стоит делать только для тренировки? Возможно, следует добавить в аргументы __keras_generator__ флаг, который будет говорить, применять аугментации или нет.
3. В качестве модели машинного обучения предлагается взять либо модель с семинара, либо какую-нибудь готовую реализацию, например из __segmentation_models__ https://github.com/qubvel/segmentation_models . Пример использования:

In [ ]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing

# prepare data
x, y = ...

preprocessing_fn = get_preprocessing('resnet34')
x = preprocessing_fn(x)

# prepare model
model = Unet(backbone_name='resnet34', encoder_weights='imagenet')
model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])

В качестве базового энкодера на семинаре использовался __resnet50__. Поэкспериментируйте с разными энкодерами, например, __VGG16__, __densenet121__. Какой дает лучшее качество?

Итак, вы обучили модель предсказывать кропы, например, размера 256x256, качество мерили тоже на кропах. Но как исходная цель - нужно уметь делать предсказания на всю картинку. Как решить проблему и есть ли она вообще? Первый приходящий в голову вариант - можно проходиться по картинке окном в 256x256 пикселей и сохранять предсказания на всю большую картинку. Но можно сделать проще, если ответить на вопрос: работает ли полностью сверточная сеть на картинках произвольного размера(с добиванием до кратности паддингами)? Почему не работали сети для классификации?

4. Напоследок, можете попробовать технику под названием __Test Time Augmentation__. Сделайте предсказание картинки, сохраните его. Попробуйте сделать flip картинки, и предсказать флипнутое изображение. Что будет если усреднить результаты этих двух предсказаний? Улучшится ли качество?

---

<h3 style="text-align: center;"><b>Ваше решение</b></h3>